# Finding Anomolies

## Setup

### Import Data

In [1]:
import os

import fiona
import geopandas as gpd
import gpxpy
import numpy as np
import pandas as pd
from multiprocess.pool import Pool
from tqdm.notebook import tqdm, trange
import warnings
import geopy.distance

warnings.filterwarnings('ignore')

fiona.drvsupport.supported_drivers['kml'] = 'rw'
fiona.drvsupport.supported_drivers['KML'] = 'rw'

storage = "/Volumes/easystore/Drones/"

flight_details = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"{storage}/gpx-with-census-data.csv", chunksize=100000, dtype=str
            ),
            desc="Loading data",
        )
    ]
)

Loading data: 0it [00:00, ?it/s]

In [30]:
geocodio_flights = pd.concat(
    [
        chunk
        for chunk in tqdm(
            pd.read_csv(
                f"{storage}/geocodio/all-flights-manifest_geocodio.csv",
                chunksize=100000,
                dtype=str,
            ),
            desc="Loading data",
        )
    ]
)
# a = pd.read_csv('../../data/outputs/outputs_cv-flight-manifest.csv')

Loading data: 0it [00:00, ?it/s]

In [8]:
flight_details["geoid"] = flight_details["geoid"].astype(str)
flight_details["len"] = flight_details["geoid"].apply(lambda x: len(x))
flight_details.loc[flight_details["len"] == 14, "geoid"] = "0" + flight_details["geoid"]


In [9]:
geocodio_flights["date"] = pd.to_datetime(geocodio_flights["date"]).dt.date


In [10]:
flight_details['altitude'] = flight_details['altitude'].astype(float)

In [11]:
flight_details['altitude'].mean()

130.50396990973857

In [91]:
# flight_ids = stable_flights.sample(1000)['id'].drop_duplicates()
potentially_low_boys = flight_details[flight_details['altitude'] <= 50]
geocodio_flights = geocodio_flights[geocodio_flights['department']=='cvpd']

flight_ids = potentially_low_boys[['id']].sample(frac=1).drop_duplicates()
flight_ids = flight_ids[flight_ids['id'].isin(geocodio_flights['id'])]['id']


### Launch Sites
- BayView Hospital = 60730131022000
- CVPD = 60730123021013
- SharpChula Medical Center = 60730133273000
- SWestern College = 60730134151000


In [92]:
potentially_low_boys = potentially_low_boys[potentially_low_boys['id'].isin(flight_ids)]
# t[['id','type','incident_id','address_map','sequence','latitude','longitude','altitude']].drop_duplicates().to_csv('./test.csv')

In [93]:
potentially_low_boys['altitude']=round(potentially_low_boys['altitude'])
potentially_low_boys=potentially_low_boys[potentially_low_boys['geoid']!='060730123021013']

In [103]:
potentially_low_boys = potentially_low_boys[potentially_low_boys['sequence'].astype(str).str.contains('2023')]

In [110]:
potentially_low_boys[['id','type','incident_id','address_map','latitude','longitude','altitude','sequence']].drop_duplicates().to_csv('./low-boys.csv',index=False)

In [104]:
compiled_flights =[] 

for r_id in potentially_low_boys['id'].drop_duplicates().values:
    
    d = gpd.read_file(f"/Volumes/easystore/Drones/flights/kml/{r_id}.kml",driver='KML')
    d['id'] = r_id
    compiled_flights.append(d)


In [109]:
paths = pd.concat(compiled_flights,ignore_index=True)
paths = paths[paths['Name']!="Home"]
paths[['geometry','id']].drop_duplicates().to_csv('./low-paths.csv',index=False)

In [111]:
paths

,Name,Description,geometry,id
1,Airdata.com,,"LINESTRING Z (-117.07164 32.61740 34.86653, -1...",d2bdc8877bdb078651008ce731d44a5a
3,Airdata.com,,"LINESTRING Z (-117.07164 32.61740 34.86653, -1...",8a33d73561e523eedeae76178b0e5229
5,Airdata.com,,"LINESTRING Z (-117.07166 32.61738 34.98082, -1...",6285b1e8aaf7455e76174c9335e6f56b
7,Airdata.com,,"LINESTRING Z (-117.07542 32.59255 11.13193, -1...",0d627ef435c0242351203e7fa247970f
9,Airdata.com,,"LINESTRING Z (-117.07168 32.61742 34.89764, -1...",6e36b515b6162150bbc19d1b65027d98
...,...,...,...,...
2799,Airdata.com,,"LINESTRING Z (-117.07167 32.61743 34.87211, -1...",6d2a6c98567541788cf6d5effd7e9cc2
2801,Airdata.com,,"LINESTRING Z (-117.07168 32.61744 34.99027, -1...",c9d3f44b57795f6babda502e37762090
2803,Airdata.com,,"LINESTRING Z (-117.07167 32.61743 34.87211, -1...",4019ff5b921f49e5cca46120cf7ba79f
2805,Airdata.com,,"LINESTRING Z (-117.07167 32.61744 34.98535, -1...",c3c86e9941fa457323fba31583193c77
